# Tarefa 4

Alunos:

* Andreza - RA: 164213
* Gil - RA: 225323
* Yan - RA: 118982

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
np.random.seed(42)

In [3]:
df = pd.read_csv("dados4.csv")
X = df.drop(['V15'], axis=1)
y = df.filter(['V15'], axis=1)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,1,22.08,11.46,2,k,bb,1.585,0,0,0,1,g,100,1213,0
1,0,22.67,7.00,2,c,bb,0.165,0,0,0,0,g,160,1,0
2,0,29.58,1.75,1,k,bb,1.250,0,0,0,1,g,280,1,0
3,0,21.67,11.50,1,j,j,0.000,1,1,11,1,g,0,1,1
4,1,20.17,8.17,2,aa,bb,1.960,1,1,14,0,g,60,159,1


## Preprocessamento

In [4]:
X = pd.get_dummies(df, columns=['V5', 'V6', 'V12'])
X.head(1)

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,1,22.08,11.46,2,1.585,0,0,0,1,100,...,0,0,0,0,0,0,0,1,0,0


In [5]:
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)
X.head(1)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.688737,-0.801052,1.347111,0.54295,-0.190906,-1.047504,-0.864196,-0.493887,1.087908,-0.488358,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079


## Métodos

In [6]:
methods = {
    'Logistic_Regression': LogisticRegression(penalty='none'),
    'Logistic_Regression_l2': LogisticRegression(penalty='l2'),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis(),
    'SVM_Linear': LinearSVC(),
    'SVM_RBF': SVC(kernel='rbf'),
    'Naive_Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'Decision_Tree': DecisionTreeClassifier(),
    'Random_Forest': RandomForestClassifier(),
    'GBM': GradientBoostingClassifier(),
}

params={
    'Logistic_Regression': None,
    'Logistic_Regression_l2': {'alpha': 10**(np.random.uniform(-3, 3, 10))},
    'LDA': None,
    'QDA': None,
    'SVM_Linear': {'C': 2**(np.random.uniform(-5, 15, 10))},
    'SVM_RBF': {'C': 2**(np.random.uniform(-5, 15, 10)),
                'gamma': 2**(np.random.uniform(-9, 3, 10))},
    'KNN': {'n_neighbors': np.random.randint(1, 301, 10)},
    'MLP': {'hidden_layer_sizes': np.arange(5, 21, 3)},
    'Decision_Tree': {'ccp_alpha': np.random.uniform(0.04, 0.0, 10)},
    'Random_Forest': {'n_estimators': [10, 100, 1000],
                      'max_features': [5, 8, 10]},
    'GBM': {'n_estimators':np.random.randint(5, 100, 10),
            'learning_rate': np.random.uniform(0.01, 0.3, 10),
            'max_depth': [2, 3]}
}

## Iterações

In [7]:
results = {
    'clf': [],
    'mean_auc_outer': [],
    'best_params': []
}

for method in methods:
    auc_outer = []
    for iteration in range(4):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        if params[method] is not None:
            clf = GridSearchCV(
                    methods[method],
                    params[method],
                    X_train,
                    y_train,
                    cv=StratifiedKFold(n_splits=3),
                    scoring='roc_auc',
                    refit=True)
        else:
            clf = GridSearchCV(
                    methods[method],
                    X_train,
                    y_train,
                    cv=StratifiedKFold(n_splits=3),
                    scoring='roc_auc',
                    refit=True)
        # get prediction on test (outer loop)
        y_pred = clf.predict(X_test)
        # calculate auc and save
        auc = roc_auc_score(y_test, y_pred)
        auc_outer.append(auc)
        break
    mean_auc = np.mean(auc_outer)
    results['clf'].append(method)
    results['mean_auc_outer'].append(mean_auc)
    results['best_params'].append(clf.best_params_)
    break


/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass scoring=     V15
309    0
131    0
44     0
70     0
265    0
..   ...
200    1
134    0
27     1
40     1
230    1

[483 rows x 1 columns] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


ValueError: Parameter grid for parameter (0) needs to be a list or numpy array, but got (<class 'pandas.core.series.Series'>). Single values need to be wrapped in a list with one element.

In [ ]:
results_df = pd.DataFrame(results)
results_df